# COVID19 - Brazil States

Install necessary packages for parallel computation:

```
pip install ipyparallel
ipcluster nbextension enable
pip install parallel-execute
```

To install for all users on JupyterHub, as root:
```
jupyter nbextension install --sys-prefix --py ipyparallel
jupyter nbextension enable --sys-prefix --py ipyparallel
jupyter serverextension enable --sys-prefix --py ipyparallel
```

start cluster at jupyter notebook interface

In [1]:
import urllib.request
import pandas as pd
import numpy as np
import cupy
from numba import cuda

ModuleNotFoundError: No module named 'pandas'

# Download data

In [3]:
import get_data
LoadData=True

if LoadData:
    get_data.get_data()

Baixando arquivos brasil.io...


In [4]:
df = pd.read_csv("data/dados_total_estados.csv")
df

,index,date,state,city,place_type,confirmed,deaths,order_for_place,is_last,popEst,city_ibge_code,confirmed_per_100k_inhabitants,death_rate
0,0,2020-06-20,AC,TOTAL,state,11263,297,96,True,881935.0,12.0,1277.07824,0.0264
1,1,2020-06-19,AC,TOTAL,state,10987,292,95,False,881935.0,12.0,1245.78342,0.0266
2,2,2020-06-18,AC,TOTAL,state,10664,287,94,False,881935.0,12.0,1209.15941,0.0269
3,3,2020-06-17,AC,TOTAL,state,10339,281,93,False,881935.0,12.0,1172.30862,0.0272
4,4,2020-06-16,AC,TOTAL,state,10003,271,92,False,881935.0,12.0,1134.21057,0.0271
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2625,203854,2020-02-29,SP,TOTAL,state,2,0,5,False,45919049.0,35.0,0.00436,0.0000
2626,203855,2020-02-28,SP,TOTAL,state,2,0,4,False,45919049.0,35.0,0.00436,0.0000
2627,203856,2020-02-27,SP,TOTAL,state,1,0,3,False,45919049.0,35.0,0.00218,0.0000
2628,203857,2020-02-26,SP,TOTAL,state,1,0,2,False,45919049.0,35.0,0.00218,0.0000


# Model

In [5]:
states=['SP','ES','MG','RJ','PE','CE','AM']
# states=['MG','ES','SP']
states

['SP', 'ES', 'MG', 'RJ', 'PE', 'CE', 'AM']

# SEAIR-D Model Equations

$$\begin{array}{l}\frac{d s}{d t}=-[\beta i(t) + \beta_2 a(t)-\mu] \cdot s(t)\\ 
\frac{d e}{d t}=[\beta i(t) + \beta_2 a(t)] \cdot s(t) -(\sigma+\mu) \cdot e(t)\\ 
\frac{d a}{d t}=\sigma e(t) \cdot (1-p)-(\gamma_2+\mu) \cdot a(t) \\
\frac{d i}{d t}=\sigma e(t) \cdot p - (\gamma + \sigma_2 + \sigma_3 + \mu) \cdot i(t)\\ 
\frac{d r}{d t}=(b + \sigma_2) \cdot i(t) + d \cdot a(t) - \mu \cdot r(t)\\
\frac{d k}{d t}=(a + \sigma_3 - \mu) \cdot d(t) + c \cdot a(t)
\end{array}$$

$$\gamma = a + b$$

$$\gamma_2 = c + d$$

$$p=0.2$$

The last equation does not need to be solve because:

$$\frac{d k}{d t}=-(\frac{d e}{d t}+\frac{d a}{d t}+\frac{d i}{d t}+\frac{d r}{d t})$$

The sum of all rates are equal to zero! The importance of this equation is that it conservates the rates.

## Parameters

$\beta$: Effective contact rate [1/min]
    
$\gamma$: from infected Recovery(+Mortality) rate $\gamma=(a+b)$ [1/min]

$\gamma_2$: from assymptomatic Recovery(+Mortality) rate $\gamma=(a+b)$ [1/min]

$a$: mortality of healed  [1/min]

$b$: recovery rate  [1/min]

$\sigma$: is the rate at which individuals move from the exposed to the infectious classes. Its reciprocal ($1/\sigma$) is the average latent (exposed) period.

$\sigma_2$: is the rate at which individuals move from the infectious to the healed classes. Its reciprocal ($1/\sigma_2$) is the average latent (exposed) period

$\sigma_3$: is the rate at which individuals move from the infectious to the dead classes. Its reciprocal ($1/\sigma_3$) is the average latent (exposed) period
    
$p$: is the fraction of the exposed which become symptomatic infectious sub-population.

$(1-p)$: is the fraction of the exposed which becomes asymptomatic infectious sub-population.

# Initial parameters

In [6]:
paramOpt=2

if paramOpt==0:
    paramFile="data/param.csv"
    version = "1"
    model = "ManualIC"

if paramOpt==1:
    paramFile="data/param_optimized_DE_Scipy.csv"    
    version = "1002"
    model = "DE_ScipyIC"
    
if paramOpt==2:
    paramFile="data/param_optimized_DE_SciPy_HistMin.csv"
    version = "1002"
    model = "DE_ScipyIC"

if paramOpt==3:
    paramFile="data/param_optimized_Yabox.csv"    
    version = "102"
    model = "YaboxIC"
    
if paramOpt==4:
    paramFile="data/param_optimized_Yabox_HistMin.csv"
    version = "102"
    model = "YaboxIC"

if paramOpt==5:
    paramFile="data/param_optimized_Brute_Force.csv"
    version = "12"
    model = "BruteForceIC"

if paramOpt==6:
    paramFile="data/param_optimized_Brute_Force_HistMin.csv"
    version = "12"
    model = "BruteForceIC"

dfparam = pd.read_csv(paramFile)
dfparam = dfparam.dropna()
dfparam

,state,start-date,prediction-range,s0,e0,a0,i0,r0,d0,START,RATIO,WCASES,WREC
0,SP,2020-03-15,200,8.864645e+06,0,0,845,300,250,100,0.15,0.361437,0.008082
1,ES,2020-04-01,200,5.279999e+05,0,0,54,250,50,100,0.15,0.318015,0.033855
2,MG,2020-04-01,200,1.301908e+06,0,0,176,250,40,100,0.15,0.358921,0.024598
3,RJ,2020-03-20,200,7.179605e+05,0,0,727,250,50,100,0.15,0.323401,0.009902
4,CE,2020-03-20,200,9.191436e+05,0,0,663,250,50,100,0.15,0.382943,0.002557
5,PE,2020-03-20,200,6.818708e+05,0,0,738,250,100,100,0.15,0.429045,0.044625
6,AM,2020-03-20,200,8.316613e+05,0,0,919,250,100,100,0.15,0.397905,0.006557


# Initial parameter optimization

In [7]:
import ray
ncpu=len(states)
ray.init(num_cpus=ncpu,num_gpus=100)

# Load solver
%reload_ext autoreload
%autoreload 2
import LearnerYabox_v2b as Learner #Yabox

ModuleNotFoundError: No module named 'ray'

In [10]:
allStates=True
cleanRecovered=False
version="7"

results=[]
if allStates:
    for state in states:        
        query = dfparam.query('state == "{}"'.format(state)).reset_index()
        parameters = np.array(query.iloc[:, 2:])[0]
        parameters
        
        f=Learner.Learner.remote(state, *parameters, cleanRecovered, version)
        result = f.train.remote() 
        results.append(result)
else:
    state= "SP"
    query = dfparam.query('state == "{}"'.format(state)).reset_index()
    parameters = np.array(query.iloc[:, 2:])[0]
    parameters
    cleanRecovered = False
    f=Learner.Learner.remote(state, *parameters, cleanRecovered, version)
    result = f.train.remote() 
    results.append(result)

# #execute all the queue with max_runner_cap at a time    
results = ray.get(results)

[autoreload of six failed: Traceback (most recent call last):
  File "/home/ats4i/anaconda3/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/ats4i/anaconda3/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 410, in superreload
    update_generic(old_obj, new_obj)
  File "/home/ats4i/anaconda3/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/home/ats4i/anaconda3/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 287, in update_class
    old_obj = getattr(old, key)
  File "/home/ats4i/anaconda3/lib/python3.7/site-packages/six.py", line 93, in __get__
    setattr(obj, self.name, result)  # Invokes __set__.
AttributeError: 'NoneType' object has no attribute 'cStringIO'
]
[autoreload of numpy.lib failed: Traceback (most recent call last):
  File "/home/ats4i/anaconda3/lib/python3.7/site-packages/IPy

[autoreload of dateutil.rrule failed: Traceback (most recent call last):
  File "/home/ats4i/anaconda3/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/ats4i/anaconda3/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 410, in superreload
    update_generic(old_obj, new_obj)
  File "/home/ats4i/anaconda3/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/home/ats4i/anaconda3/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 317, in update_class
    update_instances(old, new)
  File "/home/ats4i/anaconda3/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 280, in update_instances
    ref.__class__ = new
TypeError: __class__ assignment: 'weekday' object layout differs from 'weekday'
]
[autoreload of scipy.sparse.linalg.isolve.iterative failed: Traceback (most recent call last):
  F

RayTaskError(RuntimeError): [36mray::TemporaryActor.train()[39m (pid=108312, ip=192.168.0.104)
  File "python/ray/_raylet.pyx", line 424, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 459, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 462, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 463, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 417, in ray._raylet.execute_task.function_executor
RuntimeError: The actor with name Learner failed to be imported, and so cannot execute this method.

2020-06-21 11:20:41,424	WARNING worker.py:1090 -- Failed to unpickle actor class 'Learner' for actor ID a1651cc50100. Traceback:
Traceback (most recent call last):
  File "/home/ats4i/anaconda3/lib/python3.7/site-packages/ray/function_manager.py", line 494, in _load_actor_class_from_gcs
    actor_class = pickle.loads(pickled_class)
  File "/home/ats4i/anaconda3/lib/python3.7/site-packages/ray/cloudpickle/cloudpickle.py", line 1136, in subimport
    __import__(name)
ModuleNotFoundError: No module named 'pandas'

2020-06-21 11:20:41,425	WARNING worker.py:1090 -- Failed to unpickle actor class 'Learner' for actor ID b944ee5b0100. Traceback:
Traceback (most recent call last):
  File "/home/ats4i/anaconda3/lib/python3.7/site-packages/ray/function_manager.py", line 494, in _load_actor_class_from_gcs
    actor_class = pickle.loads(pickled_class)
  File "/home/ats4i/anaconda3/lib/python3.7/site-packages/ray/cloudpickle/cloudpickle.py", line 1136, in subimport
    __import__(name)
ModuleNotFoun

(pid=108305) 2020-06-21 11:20:41,361	ERROR function_manager.py:496 -- Failed to load actor class Learner.
(pid=108305) Traceback (most recent call last):
(pid=108305)   File "/home/ats4i/anaconda3/lib/python3.7/site-packages/ray/function_manager.py", line 494, in _load_actor_class_from_gcs
(pid=108305)     actor_class = pickle.loads(pickled_class)
(pid=108305)   File "/home/ats4i/anaconda3/lib/python3.7/site-packages/ray/cloudpickle/cloudpickle.py", line 1136, in subimport
(pid=108305)     __import__(name)
(pid=108305) ModuleNotFoundError: No module named 'pandas'
(pid=108315) 2020-06-21 11:20:41,375	ERROR function_manager.py:496 -- Failed to load actor class Learner.
(pid=108315) Traceback (most recent call last):
(pid=108315)   File "/home/ats4i/anaconda3/lib/python3.7/site-packages/ray/function_manager.py", line 494, in _load_actor_class_from_gcs
(pid=108315)     actor_class = pickle.loads(pickled_class)
(pid=108315)   File "/home/ats4i/anaconda3/lib/python3.7/site-packages/ray/clou

In [ ]:
ray.shutdown()

# Plots

In [ ]:
%matplotlib inline 
import matplotlib.pyplot as plt
import pandas as pd
#%load_ext autoreload
%reload_ext autoreload
%autoreload 2
import covid_plots

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines){
    return false;}

In [ ]:
#select districts for plotting log plot
states4Plot=['MG',
               'SP',
               'ES',
               'CE',
               'RJ']

#Choose here your options
#opt=0 all plots
#opt=1 corona log plot
#opt=2 logistic model prediction
#opt=3 bar plot with growth rate
#opt=4 log plot + bar plot
#opt=5 SEAIR-D Model
opt = 5

#versio'n to identify the png file result
# version = "1002"

#number of cases to start plotting model in log graph - real data = 100
startCase=100

In [ ]:
#plots one district or all districts
from datetime import datetime,timedelta
dfparam = pd.read_csv(paramFile)
plotStates=True

#select states for plotting
#you need to run (solve) it before plotting
if plotStates:
    states=['SP','ES','MG','RJ','PE','CE','AM']
    allStates=True

#main state for plotting log graph or SEAIRD plot
state="ES"

#if it is not SEAIRD plot, so plot one state only
if not opt==5:
    AllStates=False

if allStates:
    for state in states:
        query = dfparam.query('state == "{}"'.format(state)).reset_index()
        ratio = query['RATIO'][0]
        startCase = query['START'][0]
        startdate = query['start-date'][0]
        predict_range = query['prediction-range'][0]
        
        #calcula data máxima dos gráficos
        #100 dias é usado como máximo dos cálculos da derivada das mortes
        lastDate=df.date.max()
        maxDate= datetime.strptime(lastDate, '%Y-%m-%d') + timedelta(days = 100) #"2020-08-31"
        maxDateStr = maxDate.strftime("%Y-%m-%d")

        covid_plots.covid_plots(state, states4Plot, startdate, predict_range, \
                        startCase, 5, version, show=True, ratio=ratio, maxDate=maxDateStr,model=model)
else: 
    query = dfparam.query('state == "{}"'.format(state)).reset_index()
    ratio = query['RATIO'][0]
    startdate = query['start-date'][0]
    predict_range = query['prediction-range'][0]
    startCase = query['START'][0]
    
    #calcula data máxima dos gráficos
    #100 dias é usado como máximo dos cálculos da derivada das mortes
    lastDate=df.date.max()
    maxDate= datetime.strptime(lastDate, '%Y-%m-%d') + timedelta(days = 100) #"2020-08-31"
    maxDateStr = maxDate.strftime("%Y-%m-%d")    
    
    covid_plots.covid_plots(state, states4Plot, startdate,predict_range, \
                       startCase, opt, version, show=True, ratio=ratio, maxDate=maxDateStr,model=model)